# Numpy Compat

In [1]:
from uarray import *
import uarray
import numpy as np
import matchpy

In [2]:
@uarray.optimize
def some_fn(a, b):
    return np.multiply.outer(a, b)[10]

In [3]:
N = 10000000

In [4]:
args = [np.arange(N), np.arange(10)]

Call original function:

In [5]:
%time some_fn.__wrapped__(*args)

CPU times: user 590 ms, sys: 512 ms, total: 1.1 s
Wall time: 1.34 s


array([ 0, 10, 20, 30, 40, 50, 60, 70, 80, 90])

In [6]:
# import logging

# logging.basicConfig(level=logging.DEBUG)


In [7]:
%time some_fn(*args)

CPU times: user 50.8 ms, sys: 4.75 ms, total: 55.5 ms
Wall time: 54.7 ms


array([ 0., 10., 20., 30., 40., 50., 60., 70., 80., 90.])

## Tensorflow support

Even though we support the NumPy API, that doesn't tie us to using NumPy arrays. Instead let's add support for TensorFlow arrays and use those:

In [8]:
import tensorflow as tf

We have to define a way to take the final reduced expression and return a tensorflow graph.

We can do this by creating an empty TF array, and looping over all possible

In [9]:
class TFTensor(matchpy.Symbol):
    pass

In [10]:
class ToTFTensor(matchpy.Operation):
    name = "ToTFTensor"
    arity = matchpy.Arity(1, False)

In [11]:
class ToTFTensorInner(matchpy.Operation):
    name = "ToTFTensorInner"
    arity = matchpy.Arity(1, False)

In [12]:
register(ToTFTensor(Scalar(w.x)), lambda x: Scalar(ToTFTensor(x)))
register(ToTFTensor(Value.w.x), lambda x: ToTFTensorInner(x))

In [13]:
register(ToTFTensorInner(Value.w.x), lambda x: Content(TFTensor(tf.convert_to_tensor(x.value))))

In [23]:
register(ToTFTensor(Sequence(w.length, w.content)),lambda length, content: ToTFTensorInner(
    Sequence(
        ToTFTensor(length),
        ToGenerateTFTensor(Call(content, Content(GenerateTFTensor(lambda i: i))))
    )
))

In [24]:
class GenerateTFTensor(matchpy.Symbol):
    """
    GenerateTFTensor(lambda i: tf.Tensor(...))
    """
    pass

In [25]:
class ToGenerateTFTensor(matchpy.Operation):
    name = "ToGenerateTFTensor"
    arity = matchpy.Arity(1, False)

In [26]:
register(
    ToTFTensorInner(Sequence(Content(TFTensor.w.length), GenerateTFTensor.w.content)),
    lambda length, content: TFTensor(tf.stack(tf.map_fn(
        content.name,
        tf.range(length.name)
    ))))

In [27]:
register(Scalar(Content(GenerateTFTensor.w.x)), lambda x: x)
register(ToGenerateTFTensor(GenerateTFTensor.w.x), lambda x: x)


In [42]:
expr = uarray.OuterProduct(function(2, Multiply), uarray.Iota(uarray.scalar(100)), uarray.Iota(uarray.scalar(10)))

In [ ]:
register(ToGenerateTFTensor(Sequence(w.x, )))

In [44]:
replace(ToTFTensor((replace(expr))))

ToTFTensorInner(Sequence(Content(TFTensor(<tf.Tensor 'Const_6:0' shape=() dtype=int32>)),
                         ToGenerateTFTensor(Sequence(Value('10'),
                                                     Function(Scalar(Multiply(Content(GenerateTFTensor(<function <lambda>.<locals>.<lambda> at 0x154e348c8>)),
                                                                              Unbound('', variable_name=i40))),
                                                              Unbound('', variable_name=i40))))))

In [31]:
g_1 = tf.Graph()

In [33]:
with g_1.as_default():
    with tf.Session():
        res = uarray.replace(expr).name
        print(res.eval())

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]


In [34]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [35]:
show_graph(g_1.as_graph_def())
